<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Ronen_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from pathlib import Path
from glob import glob
import csv
import json
import pandas as pd
import os
os.chdir('/content/drive/My Drive/IDC/NLP/Assignment1/')
def preprocess():
  # TODO
 vocabulary = set()
 pathlist = glob('*.csv')
 for path in pathlist:
    with open(path, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        line_count = 0
        for text in csv_reader['tweet_text']:
            list_text = list(text)
            vocabulary.update(list_text)
            line_count += 1
        #print(line_count)
 return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [0]:
import nltk.data
import nltk.tokenize 
from collections import Counter
from collections import defaultdict,UserDict
import numpy as np

class dict_value(UserDict):
    def __init__(self, value=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.value = value

    def __missing__(self, key):
        return self.value

def find_ngrams(text, n):
    counter = Counter()
    ngram = defaultdict(lambda: defaultdict(lambda: 0))
    total_count = len(vocabulary)
    num_tokens, tokens = split_into_tokens(text.lower())
    for token in tokens:
        # sequence  = {}
        for i,_ in enumerate(token):
            if(i>=n - 1):
                word, char = token[i - (n - 1):i], token[i]
                #ngram[word] = dict_value(1/total_count)
                nchar_word = ''.join(word) + char
                counter[nchar_word] = counter.get(nchar_word, 0) + 1
                # sequence[char] = counter[nchar_word]
                ngram[word][char] = counter[nchar_word]
                #print({word:ngram[word]})
    return ngram
def token_to_unigram(token):
    token = token.strip().strip(",.!|&-_()[]<>{}/\"'").strip()

    def has_no_chars(token):
        for char in token:
            if char.isalpha():
                return False
        return True

    if len(token) == 1: # or token.isdigit(): # or has_no_chars(token):
        return None
    return token
def split_into_tokens(text):
    tokens = []
    for token in nltk.tokenize.WhitespaceTokenizer().tokenize(text):
        unigram = token_to_unigram(token)
        if unigram:
            tokens.append(unigram)
    return len(tokens), tokens
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  voc_size = len(vocabulary)
  if add_one:
       # Create a non empty dict
      model = defaultdict(lambda: defaultdict(lambda: 1/voc_size))
  else:
      model = defaultdict(lambda: defaultdict(lambda: 1e-08))
  with open(data_file_path, encoding='utf-8', newline='') as csv_file:
    csv_reader = pd.read_csv(csv_file, delimiter=',')
    corpus = []
    for tweet_text in csv_reader['tweet_text']:
            # data = '<s>' + tweet_text + '</s>'
            corpus.append(tweet_text)
    data = find_ngrams(" ".join(corpus),n)

    for word,chars in data.items():
        words_total = float(sum(chars.values()))
        total_count = words_total + len(vocabulary)
        for char, count in chars.items():
            if (add_one == True):
                p = float(count + 1)/total_count
            else:
                p = float(count)/words_total
            model[word][char] = p
  return model
n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
# #print(sum(v for v in model['abc'].values()))
# print(model['ab'])
# print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
# for j, key in enumerate(model.keys()):
#     if j < 50:
#         print("ngram:{:s} {:f}".format(key,sum(model[key].values())))  

{'@h': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f39adb5f158>, {'a': 0.22302158273381295, 'i': 0.2517985611510791, 'y': 0.03237410071942446, 'k': 0.0035971223021582736, 'e': 0.1223021582733813, 'o': 0.18345323741007194, 'j': 0.0035971223021582736, 's': 0.01079136690647482, 'u': 0.05755395683453238, 'p': 0.014388489208633094, 'b': 0.017985611510791366, 'w': 0.007194244604316547, 'c': 0.007194244604316547, 't': 0.02158273381294964, '2': 0.0035971223021582736, 'r': 0.007194244604316547, '…': 0.0035971223021582736, 'l': 0.0035971223021582736, 'q': 0.0035971223021582736, 'd': 0.0035971223021582736, 'v': 0.0035971223021582736, 'h': 0.0035971223021582736, 'f': 0.0035971223021582736, '5': 0.0035971223021582736, 'n': 0.0035971223021582736}), 'ha': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f39adb5f2f0>, {'n': 0.17027538364515452, 't': 0.2991381122556233, 'p': 0.04856001681732184, 'y': 0.004624763506411604, 's': 0.0544460794618457, 'v': 0.12423796

In [0]:
%%time
vocabulary = preprocess()
model = lm(3, vocabulary, 'en.csv', False)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
print("Sum model['Ab'].values():{:f}".format(sum(v for v in model['Ab'].values())))
print(sum(model['Ab'][v] for v in vocabulary))
print(model['Ab'])

model = lm(3, vocabulary, 'en.csv', True)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
print("Sum model['Ab'].values():{:f}".format(sum(v for v in model['Ab'].values())))
print(sum(model['Ab'][v] for v in vocabulary))
print(model['Ab'])


{'@h': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f39b0193d90>, {'a': 0.22302158273381295, 'i': 0.2517985611510791, 'y': 0.03237410071942446, 'k': 0.0035971223021582736, 'e': 0.1223021582733813, 'o': 0.18345323741007194, 'j': 0.0035971223021582736, 's': 0.01079136690647482, 'u': 0.05755395683453238, 'p': 0.014388489208633094, 'b': 0.017985611510791366, 'w': 0.007194244604316547, 'c': 0.007194244604316547, 't': 0.02158273381294964, '2': 0.0035971223021582736, 'r': 0.007194244604316547, '…': 0.0035971223021582736, 'l': 0.0035971223021582736, 'q': 0.0035971223021582736, 'd': 0.0035971223021582736, 'v': 0.0035971223021582736, 'h': 0.0035971223021582736, 'f': 0.0035971223021582736, '5': 0.0035971223021582736, 'n': 0.0035971223021582736}), 'ha': defaultdict(<function lm.<locals>.<lambda>.<locals>.<lambda> at 0x7f39b01931e0>, {'n': 0.17027538364515452, 't': 0.2991381122556233, 'p': 0.04856001681732184, 'y': 0.004624763506411604, 's': 0.0544460794618457, 'v': 0.12423796

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [0]:
def eval(n, model, data_file_path):
  data_file = pd.read_csv(data_file_path)
  count = 0
  total = 0.0
 
  for text in data_file['tweet_text'].values:
      num_tokens, tokens = split_into_tokens(text.lower())
      for data in tokens:
      #data = ['<s>'] + list(data) + ['</s>']
        for i in range(len(data) - n + 1):
            count += 1
            word, char = ''.join(data[i:i + n - 1]), data[i + n - 1]
            total += np.log2(model[word.lower()][char.lower()])

  entropy = -1 / count * total
  per = pow(2,entropy)
  #print(count)
  return round(per,3)
    

n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
test_file_name = 'it.csv'
preplexity = eval(n,model,test_file_name)
print("Preplexity evaluation : {:f}".format(preplexity))


#   return per
# n =3
# file_name = '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'
# model_b = lm_b(n, vocabulary,file_name , False)
# print(eval_b(2, model_b, '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'))

Preplexity evaluation : 27.201000


In [0]:
%%time
model = lm(3, vocabulary, 'en.csv', True)
print(eval(3, model, 'en.csv'))

22.53
CPU times: user 2.44 s, sys: 4.99 ms, total: 2.44 s
Wall time: 2.45 s


In [0]:
%%time
model = lm(3, vocabulary, 'en.csv', False)
print(eval(3, model, 'en.csv'))

8.158
CPU times: user 2.34 s, sys: 3 ms, total: 2.34 s
Wall time: 2.35 s


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
def match(n, add_one):
#  path_name = ''
#  csvglob = path_name + '*.csv'
 pathlist = glob('*.csv')
 num = len(pathlist)
 matrix = np.empty(shape=(num,num),dtype='float')
 headers_list = []
 i = 0
 for model_path in pathlist:
    head, tail = os.path.split(model_path)
    headers_list.append(tail.rstrip('.csv'))
    j = 0
    modeli = lm(n, vocabulary,model_path, add_one)
    for data_file_path in pathlist:
        #if ( i != j ):
        pp = eval(n,modeli,data_file_path)
        matrix[i][j] = pp
        #print ("Preplexity of model {:d} with data file {:d} is {:f}".format(i,j,pp))
        j = j+1
    i = i + 1
#  print(matrix)
 dataframe = pd.DataFrame(matrix,columns=headers_list,index=headers_list)
 print("passed dataframe")
 return dataframe

 print (match(3, False))

In [0]:
%%time
pd.options.display.max_columns = None
print(match(3, False))

passed dataframe
        en      tl      pt       e      fr      in      it      nl
en   8.158  29.203  43.158  40.603  57.008  36.582  27.201  30.144
tl  28.152   7.792  49.035  42.455  83.542  33.399  33.440  47.630
pt  40.700  42.515   7.446  34.020  77.133  58.918  33.539  60.664
e   32.981  37.267  30.946   8.168  55.544  44.252  25.911  42.831
fr  26.753  33.830  38.360  38.467   7.891  38.605  25.156  36.299
in  29.439  28.162  48.680  48.908  80.933   8.898  34.651  37.684
it  31.779  34.152  38.172  35.405  53.991  48.477   7.786  48.776
nl  23.567  30.015  44.867  42.933  50.293  34.263  30.285   8.251
CPU times: user 1min 37s, sys: 66.9 ms, total: 1min 37s
Wall time: 1min 37s


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [0]:
%%time
pd.options.display.max_columns = None
vocabulary = preprocess()
max_n = 4 
for n in range(4): 
  print("n = {:d} add_one:{:b}".format(n+1,False))
  print(match(n+1,False)) 
  print("n = {:d} add_one:{:b}".format(n+1,True))
  print(match(n+1,True))

n = 1 add_one:0
passed dataframe
        en      tl      pt       e      fr      in      it      nl
en  27.610  29.666  33.621  30.964  32.407  29.990  28.512  28.673
tl  30.522  26.712  33.592  31.750  36.469  27.375  29.923  30.788
pt  30.988  31.254  26.875  27.311  30.962  30.824  28.222  30.117
e   30.769  31.572  30.296  26.149  31.040  31.677  27.784  30.106
fr  30.123  32.774  30.920  29.544  27.625  32.282  27.769  29.767
in  31.138  27.639  35.379  34.401  36.785  26.413  30.550  30.595
it  30.105  30.988  30.833  28.438  30.516  31.322  25.990  29.826
nl  29.439  30.541  31.382  30.092  31.307  29.882  28.681  27.087
n = 1 add_one:1
passed dataframe
        en      tl      pt       e      fr      in      it      nl
en  27.655  29.336  30.106  29.329  30.793  29.591  27.571  28.516
tl  30.089  26.767  32.369  31.281  34.737  26.987  29.229  30.586
pt  30.487  30.756  26.940  27.087  29.978  30.326  27.560  29.768
e   30.383  31.187  27.816  26.197  29.759  31.218  27.442  29.

# **Good luck!**
max_n = 4
for n = 1 in range(max_n):
  data = match(n,False)
  print(data)
  data = match(n,True)
  print(data)
  